# Recurrent Neurons

# Basic RNNs in TensorFlow

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
n_inputs =3
n_neurons = 5

In [3]:
x0 = tf.placeholder(tf.float32, [None, n_inputs])
x1 = tf.placeholder(tf.float32, [None, n_inputs])

In [4]:
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

In [5]:
Y0 = tf.tanh(tf.matmul(x0, Wx)+ b)
Y1 = tf.tanh(tf.matmul(x0, Wy)+ tf.matmul(x1, Wx) + b)

In [6]:
init = tf.global_variables_initializer()

In [7]:
#Mini-batch: instance 0 , instance 1, instance 2, instance 3
X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]]) # t =0
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]]) # t = 1

In [8]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0,Y1], feed_dict = {x0: X0_batch,x1:X1_batch})

This mini-batch contains four instances, each with an input sequence composed of exactly two inpiuts. At the end, y0_val and y1_val contain the outputs of the network at both time steps for all neurons and all instances in the mini-batch:

In [9]:
print(Y0_val) #output at t=0

[[ 0.8673394   0.98011994  0.9973737   0.9642185   0.6319466 ]
 [-0.97549254  0.9999998   0.9999985   0.99999976 -0.50001   ]
 [-0.99997807  1.          1.          1.         -0.9511098 ]
 [-1.          1.         -0.99411535  0.99996465 -0.99998957]]


In [10]:
print(Y1_val) #output at t = 1

[[-1.          1.          1.          1.         -0.9999822 ]
 [-0.87371147 -0.99926597  1.          0.99989843 -0.9993077 ]
 [-1.          0.99995184  1.          1.         -1.        ]
 [ 0.9461739   0.9998562   0.99252063  0.89959687 -0.9999833 ]]


# Static Unrolling Through Time

The static_rnn() function creates an unrolled RNN network by chaining cells. The following code creates the exact same model as teh previous one:

In [11]:
x0 = tf.placeholder(tf.float32, [None, n_inputs])
x1 = tf.placeholder(tf.float32, [None, n_inputs])

In [12]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(
    basic_cell, [x0,x1], dtype = tf.float32)

In [13]:
Y0, Y1 = output_seqs

In [14]:
n_steps = 2

In [15]:
X = tf.placeholder(tf.float32,[None, n_steps, n_inputs])

In [16]:
X_seqs = tf.unstack(tf.transpose(X, perm=[1,0,2]))
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, reuse=True)
output_seqs, states = tf.contrib.rnn.static_rnn(
    basic_cell, X_seqs, dtype = tf.float32
)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1,0,2])

In [17]:
X_batch = np.array([
    #t=0 t=1
    [[0,1,2],[9,8,7]], # instance 0
    [[3,4,5],[0,0,0]], # instance 1
    [[6,7,8],[6,5,4]], # instance 2
    [[9,0,1],[3,2,1]], # instance 3
             
])

In [18]:
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X:X_batch})

FailedPreconditionError: Attempting to use uninitialized value rnn/basic_rnn_cell/kernel
	 [[{{node rnn/basic_rnn_cell/kernel/read}} = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](rnn/basic_rnn_cell/kernel)]]

Caused by op u'rnn/basic_rnn_cell/kernel/read', defined at:
  File "/Users/peilinjiang/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/peilinjiang/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/peilinjiang/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/peilinjiang/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/peilinjiang/tensorflow/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/Users/peilinjiang/t